In [1]:
from PIL import Image
import sys 
# change for your path to slidebook's stuff
sys.path.insert(0, 'Python/Format7')

from SBReadFile import *

from matplotlib import pyplot as plt
import numpy as np
import napari

from dask import delayed
import dask.array as da

In [10]:
theSBFileReader = SBReadFile()

# change for your image file
theSBFileReader.Open("sldy/128x_128y_5z_94t.sldy")

image = theSBFileReader.ReadImagePlaneBuf(0,0,0,0,0,True) #captureid,position,timepoint,zplane,channel,as 2d

LoadMetadata: ReadSld result:  True
CImageGroup: Load
LoadImageRecord: result  True
CImageGroup::LoadChannelRecord error
LoadChannelRecord: result  True
LoadMaks: result  True
LoadAnnotations: result  True
LoadElapsedTimes: result  True
LoadSAPositions: result  True
LoadStagePosition: result  True
LoadAuxData: result  True
LoadMetadata: theImageGroupIndex:  0  Load:  result:  True
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80


In [3]:
def read_image_plane(capture=0, position=0, timepoint=0, zplane=0, channel=0, as_2d=True):
    return theSBFileReader.ReadImagePlaneBuf(capture, position, timepoint, zplane, channel, as_2d)

# read the first file to get the shape and dtype
# ASSUMES THAT ALL FILES SHARE THE SAME SHAPE/TYPE
sample = read_image_plane()

# true is 2D or not
capture = 0
zSize=theSBFileReader.GetNumZPlanes(capture)
cSize=theSBFileReader.GetNumChannels(capture)
tSize=theSBFileReader.GetNumTimepoints(capture)

lazy_imread = delayed(read_image_plane)  # lazy reader


ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80


In [11]:
lazy_arrays = [[[lazy_imread(0,0,t,i,0,True) for i in range(zSize)] for t in range(tSize)] for c in range(2)]

In [14]:
dask_arrays = [
    [[da.from_delayed(delayed_reader, shape=sample.shape, dtype=sample.dtype)
    for delayed_reader in lazy_arrays[c][t]] for t in range(tSize)] for c in range(2)
]

In [15]:
stack = da.stack(dask_arrays)
print(stack.shape)
# in jupyter notebook the repr of a dask stack provides a useful visual:
stack

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768

ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  80
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  32848
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  65616
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  98384
ReadPlane: thePlaneSize:  32768
ReadPlane: theSeekOffset:  131152
ReadPlane: thePlaneSize:  32768

dask.array<stack, shape=(2, 94, 5, 128, 128), dtype=uint16, chunksize=(1, 94, 5, 128, 128), chunktype=numpy.ndarray>

In [16]:
napari.view_image(stack, channel_axis=0, contrast_limits=[0,255], multiscale=False)

Viewer(axes=Axes(visible=False, labels=True, colored=True, dashed=False, arrows=True), camera=Camera(center=(0.0, 63.5, 63.5), zoom=3.45859375, angles=(0.0, 0.0, 90.0), perspective=0.0, interactive=True), cursor=Cursor(position=(1.0, 1.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=3, range=((0.0, 94.0, 1.0), (0.0, 5.0, 1.0), (0.0, 128.0, 1.0), (0.0, 128.0, 1.0)), current_step=(0, 0, 0, 0), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'Image' at 0x7fd49338cc40>, <Image layer 'Image [1]' at 0x7fd4934ab610>], scale_bar=ScaleBar(visible=False, colored=False, ticks=True, position=<Position.BOTTOM_RIGHT: 'bottom_right'>, font_size=10.0, unit=None), text_overlay=TextOverlay(visible=False, color=array([0.5, 0.5, 0.5, 1. ]), font_size=10.0, position=<TextOverlayPosition.TOP_LEFT: 'top_left'>, text=''), help='', status='Ready', tooltip=T

ERROR:tornado.general:Uncaught exception in ZMQStream callback
Traceback (most recent call last):
  File "/home/laura/miniconda3/envs/sldy/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", line 431, in _run_callback
    callback(*args, **kwargs)
  File "/home/laura/miniconda3/envs/sldy/lib/python3.9/site-packages/jupyter_client/threaded.py", line 121, in _handle_recv
    msg_list = self.ioloop._asyncio_event_loop.run_until_complete(get_msg(future_msg))
  File "/home/laura/miniconda3/envs/sldy/lib/python3.9/asyncio/base_events.py", line 618, in run_until_complete
    self._check_running()
  File "/home/laura/miniconda3/envs/sldy/lib/python3.9/asyncio/base_events.py", line 580, in _check_running
    raise RuntimeError(
RuntimeError: Cannot run the event loop while another loop is running
ERROR:tornado.general:Uncaught exception in zmqstream callback
Traceback (most recent call last):
  File "/home/laura/miniconda3/envs/sldy/lib/python3.9/site-packages/zmq/eventloop/zmqstream.py", 